# TP 3

In [ ]:
# Librairies à importer pour utiliser JuMP avec le solver GLPK
using JuMP
using GLPK

# Définition de constantes pour le statut de résolution du problème
const OPTIMAL = JuMP.OPTIMAL
const INFEASIBLE = JuMP.INFEASIBLE
const UNBOUNDED = JuMP.DUAL_INFEASIBLE;

# Exercice 1 :

Une entreprise dispose de deux machines pour produire trois boissons: coca, fanta et sprite. Les quantités produites doivent être de 8 litres de coca par heure, 9 de fanta et 5 de sprite.

Pour produire un litre de boisson, chaque machine met le temps suivant, en minutes:

Production| machine 1| machine 2
-|-|-
coca | 5 | 6 
fanta | 3 | 5
sprite |     | 2 

Les dépenses de fabrication sont, par litre produit:


Dépense| machine 1| machine 2  
-|-|-
coca | 20 | 26 
fanta | 8  | 16 
sprite |      | 12 

**Q1.** Mettre sous forme d'un programme linéaire le problème qui consiste à répartir les productions sur les deux machines afin de minimiser les dépenses de production.
 
*Indication:* poser $x_{ij} = $ quantité de produit $j$ à traiter par la machine $i$ en 1h.

**Q2.** Quelle est la valeur optimale de $x_{23}$ (quantité de sprite produite par la machine $2$)?
 
**Q3.** En déduire une forme réduite du modèle.

**Q4.** Résoudre avec Julia.

In [23]:
using JuMP, GLPK

# Données fournies
C = [
    20 26
    8 16
    0 12
]

D = [8, 9, 5]

U = [
    5 6
    3 5
    0 2
]

# Création du modèle
m = Model(GLPK.Optimizer)

# Définition des variables
@variable(m, coca_m1 >= 0)  # Quantité de Coca sur la machine 1
@variable(m, fanta_m1 >= 0)  # Quantité de Fanta sur la machine 1
@variable(m, coca_m2 >= 0)  # Quantité de Coca sur la machine 2
@variable(m, fanta_m2 >= 0)  # Quantité de Fanta sur la machine 2
@variable(m, sprite_m2 >= 0)  # Quantité de Sprite sur la machine 2

# Contraintes de temps pour chaque machine
@constraint(m, 5*coca_m1 + 3*fanta_m1 <= 60)  # Machine 1
@constraint(m, 6*coca_m2 + 5*fanta_m2 + 2*sprite_m2 <= 60)  # Machine 2

# Contraintes de demande
@constraint(m, coca_m1 + coca_m2 == D[1])  # Demande de Coca
@constraint(m, fanta_m1 + fanta_m2 == D[2])  # Demande de Fanta
@constraint(m, sprite_m2 == D[3])  # Demande de Sprite

# Fonction objectif : Minimiser le coût total
@objective(m, Min, 20*coca_m1 + 8*fanta_m1 + 26*coca_m2 + 16*fanta_m2 + 12*sprite_m2)  # Coût de production

# Résolution du modèle
optimize!(m)

# Affichage des résultats
println("Solution optimale :")
println("Coca sur la machine 1 = ", value(coca_m1))
println("Fanta sur la machine 1 = ", value(fanta_m1))
println("Coca sur la machine 2 = ", value(coca_m2))
println("Fanta sur la machine 2 = ", value(fanta_m2))
println("Sprite sur la machine 2 = ", value(sprite_m2))
println("Coût total minimal = ", objective_value(m))


Solution optimale :
Coca sur la machine 1 = 6.6000000000000005
Fanta sur la machine 1 = 9.0
Coca sur la machine 2 = 1.3999999999999995
Fanta sur la machine 2 = 0.0
Sprite sur la machine 2 = 5.0
Coût total minimal = 300.4


**Q5.** Ecrire une fonction qui résout ce type de problème de manière générique.

In [28]:
using JuMP, GLPK

C = [
    20 26
    8 16
    0 12
]

D = [8 9 5]

U = [
    5 6
    3 5
    0 2
]

#C : cout, deuxieme tableau 

#D : les demandes 

#U : Utilisation des machine, premier tableau 

# Création du modèle
m = Model(GLPK.Optimizer)

# Nombre de boissons et de machines
nb_boissons = size(C, 1)
nb_machines = size(C, 2)

println("Nombre de boissons : ", nb_boissons)
println("Nombre de machines : ", nb_machines)

# Variables de décision
@variable(m, x[1:nb_machines, 1:nb_boissons] >= 0)

# Contraintes de capacité de production pour chaque machine

for machine in 1:nb_machines
    @constraint(m, sum(U[boisson, machine] * x[machine,boisson] for boisson in 1:nb_boissons) <= 60)
end

# Contraintes de demande pour chaque type de boisson
for j in 1:nb_boissons
    @constraint(m, sum(x[i,j] for i in 1:nb_machines) == D[j])
end
@constraint(m, x[1,3] ==0)
# Fonction objectif pour minimiser le coût total de production
@objective(m, Min, sum(C[j, i] * x[i,j] for i in 1:nb_machines, j in 1:nb_boissons))

# Résolution du modèle
optimize!(m)

# Affichage de la solution optimale pour chaque variable x_ij
println("Solution optimale :")
for j in 1:nb_boissons
    for i in 1:nb_machines
        println("Quantité de boisson ", j, " produite par la machine ", i, " : ", value(x[i, j]), " litres")
    end
end

# Coût total minimal de production
println("Coût total minimal de production : ", objective_value(m))


Nombre de boissons : 3
Nombre de machines : 2
Solution optimale :
Quantité de boisson 1 produite par la machine 1 : 6.6000000000000005 litres
Quantité de boisson 1 produite par la machine 2 : 1.3999999999999995 litres
Quantité de boisson 2 produite par la machine 1 : 9.0 litres
Quantité de boisson 2 produite par la machine 2 : 0.0 litres
Quantité de boisson 3 produite par la machine 1 : 0.0 litres
Quantité de boisson 3 produite par la machine 2 : 5.0 litres
Coût total minimal de production : 300.4


# Exercice 2 : 

Soit le programme linéaire $(P)$ :

$\max 120 x_1 + 100 x_2 - x_3$

* $7 x_1 + 6 x_2 \leq 12$
* $12 x_1 + 8 x_2 -x_3 \leq 20$
* $x_1,x_2,x_3 \geq 0$

**Q1.**  Ecrire le dual $(D)$ de $(P)$.

**Q2.** Résoudre $(D)$ avec Julia.

**Q3.** En déduire une solution optimale de $(P)$.

In [46]:
using JuMP, GLPK

# Création du modèle dual non générique
m_dual_non_gen = Model(GLPK.Optimizer)

# Définition des variables duales
@variable(m_dual_non_gen, y1 >= 0)  # Variable duale associée à la première contrainte du primal
@variable(m_dual_non_gen, y2 >= 0)  # Variable duale associée à la seconde contrainte du primal

# Fonction objectif du dual
@objective(m_dual_non_gen, Min, 12y1 + 20y2)

# Contraintes du dual
@constraint(m_dual_non_gen, 7y1 + 12y2 >= 120)  # Correspond à la contrainte sur x1 dans le primal
@constraint(m_dual_non_gen, 6y1 + 8y2 >= 100)   # Correspond à la contrainte sur x2 dans le primal
@constraint(m_dual_non_gen, -y2 >= -1)   # Correspond à la contrainte sur x2 dans le primal

# Résolution du modèle
optimize!(m_dual_non_gen)

# Affichage des résultats
println("Solution optimale (dual non générique) :")
println("y1 = ", value(y1), ", y2 = ", value(y2))
println("Valeur minimale de la fonction objectif (dual non générique) : ", objective_value(m_dual_non_gen))


Solution optimale (dual non générique) :
y1 = 15.428571428571429, y2 = 1.0
Valeur minimale de la fonction objectif (dual non générique) : 205.14285714285714


In [48]:
using JuMP, GLPK

# Création du modèle
m = Model(GLPK.Optimizer)

# Définition des variables duales
@variable(m, lambda[1:2] >= 0)

# Fonction objectif
@objective(m, Min, 12*lambda[1] + 20*lambda[2])

# Contraintes
@constraint(m, 7*lambda[1] + 12*lambda[2] >= 120)
@constraint(m, 6*lambda[1] + 8*lambda[2] >= 100)
@constraint(m, - lambda[2] >= -1)

# Résolution du modèle
optimize!(m)


# Affichage de la solution optimale
println("Solution optimale (dual générique) :")
println("lambda[1] = ", value(lambda[1]))
println("lambda[2] = ", value(lambda[2]))
println("Valeur minimale de la fonction objectif (dual générique) : ", objective_value(m))


Solution optimale (dual générique) :
lambda[1] = 15.428571428571429
lambda[2] = 1.0
Valeur minimale de la fonction objectif (dual générique) : 205.14285714285714
